[맵리듀스 이해하기](http://blog.acronym.co.kr/312)  
[맵리듀스 연동](http://wiki.gurubee.net/pages/viewpage.action?pageId=28117503)  
하기 내용은 상기 링크를 정리한것.

# 맵리듀스(MapReduce) 개요

맵리듀스(MapReduce)는 기존 하드웨어를 활용한 분산 프로그래밍 모델로서,  
대용량의 데이터를 빠르고 안전하게 처리하기 위해 만들어젔다.  
2014년 OSDI컴퍼런스에서 구글이 "MapReduce: Sim plified Data Processing on Large Cluster"논문 발표 이후 관심받기 시작했다. 
그리고 오픈소스 루씬(Lucene)의 개발자인 더그 커팅(Doug Cutting)이 하둡(Hadoop)을 만들면서 알려지기 시작했다.  

## MapReduce 개념
맵리듀스 이해를 위해 기존 프로그래밍 방식을 생각해본다.  
1. 데이터를 가져오고
2. 가져온 데이터를 처리
3. 처리한 데이터를 저장한다.
즉, 데이터를 가져와서 중앙에서 처리하고 다시 저장하는 구조이다.  
그런 구조로 인해 많은 비용이 든다.  


따라서 적은 비용으로 처리할 방법이 필요했고 그래서 나온것이 MapReduce이다.  
MapReduce는 Map과 Reduce의 단계로 구성된다.  
## MapReduce 이해
Map 단계와 Reduce단계는 모두 입/출력으로 키-값(Key-Value) 쌍을 가진다.
Key와 Value는 개발자의 선택이며, map함수와 reduce함수도 개발자가 직접 작성해야 된다.  

## MapReduce 구조
![](http://wiki.gurubee.net/download/attachments/28117503/Figure_7-1.png)

### Map, Key
Map함수는 Key:줄번호(라인번호) 와 Value: 문장(라인)으로 구성된다.  
Key는 줄번호 뿐만 아니라, 파일의 각 줄의 시작 오프셋(offset)이 된다.  
  
Map은 각 문장을 공백으로 분류하고 단어와 글자 수를 Key-Value형태로 가공한후 정렬과 병합을 한다.  
여기까지가 Local에서 일어나는 Map의 과정이다.  
  
Reduce함수는 "단어-글자수 목록"을 입력으로 받아서, 각 단어들의 개수를 취합해 최종 결과를 제공한다.  


### Splitting, Shuffling
MapReduce는 대용량의 입력 파일을 Split(분리)뒤 Map 함수에 적용되고,  
Reduce함수에 전달하기 위해 Shuffle, Sort이 실행된후 최종적으로 기록이 된다.<br>
<br>

![](https://t1.daumcdn.net/cfile/tistory/2133764B54F929D108)

하둡에서 최초로 입력된 파일은 먼저 Splitting되어 HDFS에 저장된다.  
대용량 파일을 한번에 처리할 수 없기에, 적절한 크리로 처리한후 MapReduce처리를 하는 것이다.  
그러나 너무 작게 Splitting이 될경우 분활 관리 및 태스트 생성의 오버헤드가 있어 역효과가 날수 있다.  
보통 Hadoop에서는 64MB 이상의 Splitting을 권장하고 있다.  
클라우데라의 CDH에서는 기본값이 128MB로 설정되어 있다.  
  
아래의 그림처럼 Splitting은 Map 함수의 결과를 취합하기 위해 Reduce 함수로 데이터를 전달하는 역활을 한다.  
그전에 Map함수의 결과에 대해서 Sort와 Merge과정이 벌어지고 각 서버에 나뉜 데이터를 키를 중심으로 모아 Reduce함수에 전달한다.  
이렇게 Reduce 함수로 데이터를 전달하는 것을 위의 그림과 같이 Shuffling이라고 한다.  

![](https://t1.daumcdn.net/cfile/tistory/266D5E4B53F9CFD503)

### Partitioner, Combiner
마지막으로 Partitioner과 Combiner에 대해 알아본다.  
  
Partitioner는 Map 함수의 결과를 각 파티션으로 나눠 저장하는 역활을 한다. 
Partitioner을 나누는 기준은 Key이며, 기본적으로 Key에 Hash함수를 적용하여 처리한다.  
위 그림에도 Map결과를 Buffer Memory에 저장하고, 디스크에 저장할 때Pertition과 Sort등의 작업이 일어나는 것을 확인할수 있다.
  
Combiner는 Reducer와 동일하게 사용되기에 "Local Reuce"라고 부르기도 한다.  
그렇다면 Combiner는 왜 필요한 것일까?  
MapReduce는 대용량을 처리하기 위해 만들어 졌고 Map함수는 Local에서 실행하지만 Reduce 함수로 데이터를 전달하는 것은 네트워크를 이용하기 때문이다.  
따라서 Local에서 Reduce를 처리한다면, 네트워크 비용이 매우 감소 할 것이다.

<br>
Reduce 함수를 Combiner로 사용하기 위해서는 Reduce함수의 입력과 출력의 Type이 같아야 한다.  
그렇지 않을 경우에는 별도의 Combiner를 구현해서 활용할 수 있다.  
<br>
<br>
![](https://t1.daumcdn.net/cfile/tistory/2543654353F9D3CA02)

Combiner적용에 차이는 위의 그림으로 부터 확인할 수 있다.  
상단의 그림에서는 매우 적은양의 데이터가 분리 되지만 많은 데이터에 Combiner를 적용한다면 비용의 절감을 무시할 수 없을 것이다.